In [1]:
from __future__ import annotations
import os
from dotenv import load_dotenv
from services import api_groups
import openapi_client
from openapi_client.rest import ApiException
import time

from services import actions as act
import state
from session import init_db

configuration = openapi_client.Configuration(access_token=os.getenv("BEARER_TOKEN"))
apicl = openapi_client.ApiClient(configuration)
agents = openapi_client.AgentsApi(apicl)
contracts = openapi_client.ContractsApi(apicl)
systems = openapi_client.SystemsApi(apicl)
fleet = openapi_client.FleetApi(apicl)

In [2]:
from rich import print as rprint
from rich.pretty import Pretty

load_dotenv()

def pretty(obj):
    """
    Recursively pretty-print any object with color.
    Works with Pydantic/OpenAPI models, dicts, lists, etc.
    """
    # Convert Pydantic/OpenAPI model to dict if possible
    try:
        data = obj.dict(by_alias=True, exclude_none=True)
    except AttributeError:
        data = obj
    rprint(Pretty(data))

me = agents.get_my_agent()
my_system = '-'.join(me.data.headquarters.split('-')[:2])
print("My system :", my_system)

waypoints_list = systems.get_system_waypoints(my_system)
mine_site = waypoints_list.data[1].symbol
rprint("Mine site: ",mine_site)

My system : X1-PR33


Mine site:  X1-PR33-FE5D

In [4]:
ships = act.list_ships()
print("Frigate is :", ships[0].symbol)
frigate = ships[0].symbol

Frigate is : CRACKA-1


In [5]:
act.orbit(frigate)

DockShip200ResponseData(nav=ShipNav(system_symbol='X1-PR33', waypoint_symbol='X1-PR33-FE5D', route=ShipNavRoute(destination=ShipNavRouteWaypoint(symbol='X1-PR33-FE5D', type=<WaypointType.ENGINEERED_ASTEROID: 'ENGINEERED_ASTEROID'>, system_symbol='X1-PR33', x=-6, y=-25), origin=ShipNavRouteWaypoint(symbol='X1-PR33-A1', type=<WaypointType.PLANET: 'PLANET'>, system_symbol='X1-PR33', x=-22, y=9), departure_time=datetime.datetime(2025, 9, 4, 20, 36, 55, 589000, tzinfo=TzInfo(UTC)), arrival=datetime.datetime(2025, 9, 4, 20, 37, 36, 589000, tzinfo=TzInfo(UTC))), status=<ShipNavStatus.IN_ORBIT: 'IN_ORBIT'>, flight_mode=<ShipNavFlightMode.CRUISE: 'CRUISE'>))

In [12]:
fleet.extract_resources(frigate)


ExtractResources201Response(data=ExtractResources201ResponseData(extraction=Extraction(ship_symbol='CRACKA-1', var_yield=ExtractionYield(symbol=<TradeSymbol.QUARTZ_SAND: 'QUARTZ_SAND'>, units=3)), cooldown=Cooldown(ship_symbol='CRACKA-1', total_seconds=80, remaining_seconds=79, expiration=datetime.datetime(2025, 9, 4, 20, 52, 33, 866000, tzinfo=TzInfo(UTC))), cargo=ShipCargo(capacity=40, units=5, inventory=[ShipCargoItem(symbol=<TradeSymbol.COPPER_ORE: 'COPPER_ORE'>, name='Copper Ore', description='A valuable ore used in the production of copper and other alloys. Copper ore is an essential component in the production of electronic components and wiring.', units=2), ShipCargoItem(symbol=<TradeSymbol.QUARTZ_SAND: 'QUARTZ_SAND'>, name='Quartz Sand', description='High-purity quartz sand used in the production of glass and ceramics.', units=3)]), modifiers=[], events=[]))

In [ ]:
scanned_wps = fleet.create_ship_waypoint_scan(frigate)
pretty(scanned_wps)

CreateShipWaypointScan201Response(data=CreateShipWaypointScan201ResponseData(cooldown=Cooldown(ship_symbol='CRACKA-1', total_seconds=80, remaining_seconds=79, expiration=datetime.datetime(2025, 9, 4, 20, 56, 24, 231000, tzinfo=TzInfo(UTC))), waypoints=[ScannedWaypoint(symbol='X1-PR33-A1', type=<WaypointType.PLANET: 'PLANET'>, system_symbol='X1-PR33', x=-22, y=9, orbitals=[WaypointOrbital(symbol='X1-PR33-A2'), WaypointOrbital(symbol='X1-PR33-A3'), WaypointOrbital(symbol='X1-PR33-A4')], faction=WaypointFaction(symbol=<FactionSymbol.AEGIS: 'AEGIS'>), traits=[WaypointTrait(symbol=<WaypointTraitSymbol.VOLCANIC: 'VOLCANIC'>, name='Volcanic', description='A volatile world marked by intense volcanic activity, creating a hazardous environment with the potential for valuable resource extraction, such as rare metals and geothermal energy.'), WaypointTrait(symbol=<WaypointTraitSymbol.OUTPOST: 'OUTPOST'>, name='Outpost', description='A small, remote settlement providing essential services and a saf